In [ ]:
# Bank churn classifier (neural network using scikit-learn MLP)
# Satisfies: read dataset, split features/target, normalize, build model,
# implement improvements (handle class imbalance by upsampling minority; enable early stopping),
# print accuracy and confusion matrix.

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import resample

RND = 42

# 1. Read dataset (local preferred; fallback to public mirror)
# path = "Churn_Modelling.csv"
df = pd.read_csv("path")


# 2. Feature/target split and basic preprocessing
if 'Exited' not in df.columns:
    raise RuntimeError("Dataset must contain 'Exited' column (target).")

df = df.copy()
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df = pd.get_dummies(df, columns=['Geography'], drop_first=True)
df = df.drop(columns=[c for c in ['CustomerId', 'Surname', 'RowNumber'] if c in df.columns])

X = df.drop(columns=['Exited'])
y = df['Exited'].astype(int)

# stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RND, stratify=y
)

# 3. Normalize train and test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Identify improvement: class imbalance handling -> upsample minority class in training set
# Combine X_train and y_train to perform resampling
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['Exited'] = y_train.values

majority = train_df[train_df['Exited'] == 0]
minority = train_df[train_df['Exited'] == 1]

if len(minority) == 0:
    # if no minority present (unexpected), skip resampling
    X_train_res = X_train
    y_train_res = y_train
else:
    minority_upsampled = resample(minority,
                                  replace=True,
                                  n_samples=len(majority),
                                  random_state=RND)
    train_balanced = pd.concat([majority, minority_upsampled])
    train_balanced = train_balanced.sample(frac=1, random_state=RND).reset_index(drop=True)
    y_train_res = train_balanced['Exited'].astype(int)
    X_train_res = train_balanced.drop(columns=['Exited']).values

# 4. Initialize and build the model (MLP with early stopping)
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=20,
    tol=1e-4,
    random_state=RND
)

# Train on the (possibly) resampled training set
mlp.fit(X_train_res, y_train_res)

# 5. Evaluate: print accuracy score and confusion matrix
y_pred = mlp.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print("Confusion matrix:")
print(cm)

Accuracy: 0.7820
Confusion matrix:
[[1301  292]
 [ 144  263]]


1. Importing Libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import resample


Explanation:

os → For handling file paths (not heavily used here).

numpy → For numerical operations.

pandas → For loading and manipulating the dataset.

train_test_split → To split data into training and testing sets.

LabelEncoder → Convert categorical text labels (like gender) into numbers.

StandardScaler → Normalize features (mean=0, std=1) for faster neural network training.

MLPClassifier → Multi-layer Perceptron (neural network) classifier.

accuracy_score, confusion_matrix → Evaluate model performance.

resample → Used to handle class imbalance (upsample minority class).

2. Set random seed
RND = 42


Explanation:

RND → Ensures reproducibility. Random processes (like shuffling data) give the same result each time.

3. Load the dataset
df = pd.read_csv("path")


Explanation:

Reads the Bank Churn dataset CSV into a pandas dataframe.

Each row → a customer

Columns → customer details like CreditScore, Gender, Geography, Exited (target).

4. Basic preprocessing and feature/target split
if 'Exited' not in df.columns:
    raise RuntimeError("Dataset must contain 'Exited' column (target).")

df = df.copy()
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df = pd.get_dummies(df, columns=['Geography'], drop_first=True)
df = df.drop(columns=[c for c in ['CustomerId', 'Surname', 'RowNumber'] if c in df.columns])

X = df.drop(columns=['Exited'])
y = df['Exited'].astype(int)


Explanation:

Ensure Exited column exists → target variable (1 = left, 0 = stayed).

Gender → Convert 'Male'/'Female' to 0/1.

Geography → Convert categorical countries into dummy variables (one-hot encoding), drop first to avoid redundancy.

Drop irrelevant columns: CustomerId, Surname, RowNumber → do not help in prediction.

X → All features except Exited.

y → Target variable (Exited).

5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RND, stratify=y
)


Explanation:

Split data into 80% training, 20% testing.

stratify=y → Maintains the proportion of churners/non-churners in train and test sets.

6. Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Explanation:

Neural networks work better if features are on the same scale.

fit_transform() → Compute mean/std on train data and normalize it.

transform() → Apply the same normalization to test data.

7. Handle class imbalance (upsampling minority class)
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['Exited'] = y_train.values

majority = train_df[train_df['Exited'] == 0]
minority = train_df[train_df['Exited'] == 1]

if len(minority) == 0:
    X_train_res = X_train
    y_train_res = y_train
else:
    minority_upsampled = resample(minority,
                                  replace=True,
                                  n_samples=len(majority),
                                  random_state=RND)
    train_balanced = pd.concat([majority, minority_upsampled])
    train_balanced = train_balanced.sample(frac=1, random_state=RND).reset_index(drop=True)
    y_train_res = train_balanced['Exited'].astype(int)
    X_train_res = train_balanced.drop(columns=['Exited']).values


Explanation:

Class imbalance → fewer churners than non-churners. Neural network may ignore minority class.

Combine X_train and y_train into one dataframe for resampling.

Separate majority class (Exited=0) and minority class (Exited=1).

resample() → Upsample minority class with replacement to match majority class size.

train_balanced → Shuffled balanced training data.

X_train_res → Features for training after resampling.

y_train_res → Target for training after resampling.

8. Build and train MLP (neural network)
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=20,
    tol=1e-4,
    random_state=RND
)

mlp.fit(X_train_res, y_train_res)


Explanation:

hidden_layer_sizes=(64,32) → Two hidden layers: first with 64 neurons, second with 32.

activation='relu' → Non-linear activation function.

solver='adam' → Optimizer for weight updates.

max_iter=500 → Maximum number of epochs.

early_stopping=True → Stop training if validation score stops improving.

n_iter_no_change=20 → Number of epochs to wait for improvement before stopping.

tol=1e-4 → Minimum improvement threshold.

fit() → Train the model on the (balanced) training set.

9. Evaluate the model
y_pred = mlp.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print("Confusion matrix:")
print(cm)


Explanation:

predict() → Predict churn/not-churn on test set.

accuracy_score → Overall proportion of correct predictions.

confusion_matrix → Shows counts of True Negatives, False Positives, False Negatives, True Positives.

Example output of cm:

[[TN  FP]
 [FN  TP]]


Helps identify how well the model predicts churners vs non-churners.